In [2]:
import commentScraper
from googleapiclient.discovery import build 
import sys
from dotenv import load_dotenv
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

c:\Users\araf7\OneDrive\Desktop\Youtube Comment Sentiment Analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Google API Key
load_dotenv()
api_key = os.getenv("YOUTUBE_API_KEY")

# User input
if len(sys.argv) != 2:
    print("Add a video ID after the script name")
    sys.exit(1)
    
vid_id = sys.argv[1]

yt_client = build("youtube", "v3", developerKey=api_key)

In [4]:
comment_df = commentScraper.comments_df(yt_client,"wGAhlQ8vGfI").sort_values(by="Number of Likes", ascending=False)[:100].drop_duplicates()

In [5]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('model', local_files_only=True)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [6]:
text = comment_df["Comment"].tolist()

In [7]:
# Run inference
inputs = tokenizer(text, return_tensors="pt",padding=True, truncation=True)

with torch.no_grad():
    logits = model(**inputs).logits
    prediction = torch.argmax(logits, dim=1)

comment_df["Predicted_class"] = prediction.numpy()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [29]:
comment_df["Predicted_class"].value_counts()

Predicted_class
0    45
1    35
2    20
Name: count, dtype: int64